In [1]:
from fa_ensemble import FiniteAggregationEnsemble
from torchvision.datasets import CIFAR10, MNIST
from torchvision import transforms
import torch
from torch import nn, optim
import random
import numpy
from tqdm import tqdm
import ssl
from models.NetworkInNetwork import NetworkInNetwork
from threading import Thread

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
trainset = MNIST(root='./datasets/MNIST', train=True, download=True, transform=transforms.Compose([
    transforms.RandomCrop(28, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))
testset = MNIST(root='./datasets/MNIST', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 41292240.02it/s]


Extracting ./datasets/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/MNIST/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1218951.00it/s]


Extracting ./datasets/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/MNIST/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 10259718.25it/s]


Extracting ./datasets/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/MNIST/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4680719.60it/s]

Extracting ./datasets/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/MNIST/MNIST/raw



In [4]:
def train_base_model(partition_number, train_subset):
    seed = partition_number
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    curr_lr = 0.1
    epochs = 200
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )

    trainloader = torch.utils.data.DataLoader(train_subset, batch_size=128, shuffle=True, num_workers=1)
    print("subset has ", len(train_subset), "data points")

    ssl._create_default_https_context = ssl._create_unverified_context
    net = NetworkInNetwork({'num_classes':10, 'num_inchannels': 1})

    net = net.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(net.parameters(), lr=curr_lr, momentum=0.9, weight_decay=0.0005, nesterov= True)

    # Training
    net.train()
    for epoch in tqdm(range(epochs)):
        for (inputs, targets) in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        if (epoch in [60,120,160]):
            curr_lr = curr_lr * 0.2
            for param_group in optimizer.param_groups:
                param_group['lr'] = curr_lr


    net.eval()
    nomtestloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=1)
    correct = 0
    total = 0
    for (inputs, targets) in nomtestloader:
        inputs, targets = inputs.to(device), targets.to(device)
        with torch.no_grad():
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
        break
    acc = 100.*correct/total
    print(f'Estimated accuracy for base model {partition_number}: {str(acc)}%')

    return net


In [5]:
ensemble = FiniteAggregationEnsemble("drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1", trainset, testset, 10, 1200)

In [6]:
for i in range(800):
  ensemble.train_base_model(i, train_base_model)

Base model 0 already exists
Base model 1 already exists
Base model 2 already exists
Base model 3 already exists
Base model 4 already exists
Base model 5 already exists
Base model 6 already exists
Base model 7 already exists
Base model 8 already exists
Base model 9 already exists
Base model 10 already exists
Base model 11 already exists
Base model 12 already exists
Base model 13 already exists
Base model 14 already exists
Base model 15 already exists
Base model 16 already exists
Base model 17 already exists
Base model 18 already exists
Base model 19 already exists
Base model 20 already exists
Base model 21 already exists
Base model 22 already exists
Base model 23 already exists
Base model 24 already exists
Base model 25 already exists
Base model 26 already exists
Base model 27 already exists
Base model 28 already exists
Base model 29 already exists
Base model 30 already exists
Base model 31 already exists
Base model 32 already exists
Base model 33 already exists
Base model 34 already ex

In [7]:
ensemble.eval("softmax_median")

testset predictions already computed, using those...
Certificates already computed, using those...
Base classifier accuracy: 77.28179166666666
Clean Accuracy: 95.63000000000001%
Median Certified Radius: 496


In [8]:
ensemble.eval("logit_median")

testset predictions already computed, using those...
Certificates already computed, using those...
Base classifier accuracy: 77.28179166666666
Clean Accuracy: 95.39999999999999%
Median Certified Radius: 397


In [9]:
ensemble.eval(mode="label_voting")

testset predictions already computed, using those...
Certificates already computed, using those...
Base classifier accuracy: 77.28179166666666
Clean Accuracy: 95.85000000000001%
Median Certified Radius: 451


In [10]:
ensemble.eval(mode="label_runoff")

testset predictions already computed, using those...
Certificates already computed, using those...
Base classifier accuracy: 77.28179166666666
Clean Accuracy: 95.53%
Median Certified Radius: 467


In [11]:
student = NetworkInNetwork({'num_classes':10, 'num_inchannels': 1})
ensemble.distill(student, mode='logit_median', lr=1e-3, epochs=10)

drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_logit_median2305843009213694100.pkl
Student already trained with those parameters at drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_logit_median2305843009213694100.pkl, loading it in instead of training again...
Evaluating Student


  0%|          | 0/79 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 79/79 [00:11<00:00,  6.83it/s]

Accuracy for student: 94.97%


In [12]:
student = NetworkInNetwork({'num_classes':10, 'num_inchannels': 1})
ensemble.distill(student, mode='softmax_median', lr=1e-3, epochs=10)

drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_softmax_median2305843009213694100.pkl
Student already trained with those parameters at drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_softmax_median2305843009213694100.pkl, loading it in instead of training again...
Evaluating Student


100%|██████████| 79/79 [00:11<00:00,  6.91it/s]

Accuracy for student: 95.97%


In [13]:
student = NetworkInNetwork({'num_classes':10, 'num_inchannels': 1})
ensemble.distill(student, mode='label_voting', lr=1e-3, epochs=10)

drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_label_voting2305843009213694100.pkl
Student already trained with those parameters at drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_label_voting2305843009213694100.pkl, loading it in instead of training again...
Evaluating Student


100%|██████████| 79/79 [00:11<00:00,  6.90it/s]

Accuracy for student: 96.3%


In [14]:
student = NetworkInNetwork({'num_classes':10, 'num_inchannels': 1})
ensemble.distill(student, mode='label_runoff', lr=1e-3, epochs=10)

drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_label_runoff2305843009213694100.pkl
Student already trained with those parameters at drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_label_runoff2305843009213694100.pkl, loading it in instead of training again...
Evaluating Student


100%|██████████| 79/79 [00:09<00:00,  8.23it/s]

Accuracy for student: 95.85%


In [15]:
student = NetworkInNetwork({'num_classes':10, 'num_inchannels': 1})
ensemble.distill(student, mode='logit_median', lr=1e-2, epochs=10)

drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_logit_median23058430092136940100.pkl
Student already trained with those parameters at drive/MyDrive/763Proj/ensembles/mnist_benchmark_k=1200_d=1/students/student_logit_median23058430092136940100.pkl, loading it in instead of training again...
Evaluating Student


100%|██████████| 79/79 [00:09<00:00,  8.00it/s]

Accuracy for student: 94.71%
